## References

see README.md

In [10]:
import os
import itertools


import datetime

import editdistance
import numpy as np

import pylab
import matplotlib.pyplot as plt
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model
from keras.layers.recurrent import GRU
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
from keras.preprocessing import image
import keras.callbacks

In [11]:


np.random.seed(55)

In [12]:
import generator_text_image as fake



In [ ]:
# verify what we have with existing generator
def lets_understand_original_generator():
    
    img_w = 128
    img_h = 64
    pool_size = 2
    words_per_epoch = 16000
    val_split = 0.2
    val_words = int(words_per_epoch * (val_split))
    minibatch_size = 32
    
    print("INPUT to generator:")
    print("-------------------")
    print()


    fdir = os.path.dirname(get_file('wordlists.tgz',
                                    origin='http://www.mythic-ai.com/datasets/wordlists.tgz', untar=True))

    print("img_w", img_w)
    print("img_h", img_h)
    print("pool_size", pool_size)
    print("words_per_epoch", words_per_epoch)
    print("val_words", val_words)
    img_gen = fake.TextImageGenerator(monogram_file=os.path.join(fdir, 'wordlist_mono_clean.txt'),
                                 bigram_file=os.path.join(fdir, 'wordlist_bi_clean.txt'),
                                 minibatch_size=minibatch_size,
                                 img_w=img_w,
                                 img_h=img_h,
                                 downsample_factor=(pool_size ** 2),
                                 val_split=words_per_epoch - val_words
                                 )
    
    print()
    print("OUTPUT from generator:")
    print("----------------------")
    print()
    print("img_gen.get_output_size()", img_gen.get_output_size())
    print("img_gen.absolute_max_string_len", img_gen.absolute_max_string_len)
    img_gen.on_train_begin()
    
    #temp = img_gen.next_train() # output is dummy data according to comments
    
    #for i in temp:
    i =  img_gen.get_batch(1, 32, train=True)
    temp_input, temp_output_is_dummy = i

    print()
    print('this corresponds to cnn_rnn_model Input(name=the_input')
    print("the_input is a set of images ", temp_input['the_input'].shape)

    print()
    print("source_str for visualization", temp_input['source_str'][:8])

    print()
    print("the next are inputs to the model for ctc calculation")
    print()
    print("this corresponds to cnn_rnn_model Input(name='the_labels'")
    print("the_labels are char index from a, ex:t=19")
    print(temp_input['the_labels'][:2])

    print()
    print("this corresponds to cnn_rnn_model Input(name='input_length'")
    print('input_length of ctc is 2 short, see ctc_drop_first_2')
    print(temp_input['input_length'][:2])

    print()
    print("this corresponds to cnn_rnn_model Input(name='label_length'")
    print('label_length is number of chars in word')
    print(temp_input['label_length'][:8])

   #     break
    


In [ ]:
lets_understand_original_generator()

# IAM based generator

In [13]:
import PIL
import numpy as np
import generator_iam_words as IAM
import generator_german_words as GER

In [ ]:
# verify what we have with existing generator
def lets_understand_IAM():
    
    img_w = 128
    img_h = 64
    pool_size = 2
    words_per_epoch = 16000
    val_split = 0.2
    val_words = int(words_per_epoch * (val_split))
    minibatch_size = 32
    
    print("INPUT to generator:")
    print("-------------------")
    print()
    
   
    print("img_w", img_w)
    print("img_h", img_h)
    print("pool_size", pool_size)
    print("words_per_epoch", words_per_epoch)
    print("val_words", val_words)
    g = IAM.IAM_Word_Generator(minibatch_size = 32, img_w = 128, img_h = 64, downsample_factor=4, absolute_max_string_len=16)
 
    g.on_train_begin()
    x = g.get_batch(-1, 32, True)
  
    return x
    


In [ ]:
x = lets_understand_IAM()
input, output = x

the_input = input["the_input"]
print(the_input.shape)

print(input["source_str"])

print(input['the_labels'])

print(input['input_length'])

print(input['label_length'])





# Three Generators Side by Side

In [14]:
def side_by_side():
    img_w = 128
    img_h = 64
    pool_size = 2
    words_per_epoch = 16000
    val_split = 0.2
    val_words = int(words_per_epoch * (val_split))
    minibatch_size = 32
    
    fdir = os.path.dirname(get_file('wordlists.tgz',
                                    origin='http://www.mythic-ai.com/datasets/wordlists.tgz', untar=True))

    # original
    img_gen = fake.TextImageGenerator(monogram_file=os.path.join(fdir, 'wordlist_mono_clean.txt'),
                                 bigram_file=os.path.join(fdir, 'wordlist_bi_clean.txt'),
                                 minibatch_size=minibatch_size,
                                 img_w=img_w,
                                 img_h=img_h,
                                 downsample_factor=(pool_size ** 2),
                                 val_split=words_per_epoch - val_words
                                 )
    img_gen.on_train_begin()
    orig_batch =  img_gen.get_batch(1, 32, train=True)
    
    # IAM based
    iam_gen = IAM.IAM_Word_Generator(minibatch_size = 32, img_w = 128, img_h = 64, downsample_factor=4, absolute_max_string_len=16)
    iam_gen.on_train_begin()
    iam_batch = iam_gen.get_batch(iam_gen.words_id_text_list, 32, True)
    
    # German Chruch record
    ger_gen = GER.German_Word_Generator(minibatch_size = 32, img_w = 128, img_h = 64, downsample_factor=4, absolute_max_string_len=16)
    ger_gen.on_train_begin()
    ger_batch = ger_gen.get_batch(ger_gen.words_id_text_list, 32, True)
    
    print("len of get_batch, orig vs IAM vs ger,", 
          len(orig_batch), "vs", 
          len(iam_batch), "vs", len(ger_batch))
   
    input_orig, output_orig = orig_batch
    input_iam, output_iam = iam_batch
    input_ger, output_ger = ger_batch

    the_input_orig = input_orig["the_input"]
    the_input_iam = input_iam["the_input"]
    the_input_ger = input_ger["the_input"]
    
    print("the_input ie: image", the_input_orig.shape, "vs", the_input_iam.shape, "vs", the_input_ger.shape)
    print("image[0,0,0,0]")

    print("source_str, first 5", input_orig["source_str"][:5], "vs", input_iam["source_str"][:5], "vs", input_ger["source_str"][:5])

    print("the_labels, first 2", input_orig['the_labels'][:2], "vs", input_iam['the_labels'][:2], "vs", input_ger['the_labels'][:2])

    print("input_length, first 2", input_orig['input_length'][:2], "vs", input_iam['input_length'][:2], "vs", input_ger['input_length'][:2])

    print("label_length, first 2", input_orig['label_length'], "vs", input_iam['label_length'], "vs", input_ger['label_length'])
    

In [15]:
side_by_side()

on_train_begin
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
on_train_begin
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
shape (128, 64, 1)
s